# Credits
This borrows, at times verbatim, from various notebooks/blogs on the internet which themselves borrow heavily from the [official PyTorch tutorials](https://github.com/pytorch/tutorials).

# Getting started with PyTorch

## Tensors
Construct a 5x3 matrix, uninitialized

In [ ]:
import torch

x = torch.Tensor(5,3)
print(x)

### DIY
Use official PyTorch documentation to perform the following
+ Make a tensor of size (2, 17)
+ Make a torch.FloatTensor of size (3, 1)
+ Make a torch.LongTensor of size (5, 2, 1) then fill it all with 7s
+ Make a torch.ByteTensor of size (5,) and then make it [0, 1, 1, 1, 0]

## Operations

We can perform addition operation in quite a few different ways

In [ ]:
y = torch.rand(5, 3)
print(x + y)

In [ ]:
print(torch.add(x, y))

In [ ]:
y.add_(x)
print(y)

### DIY
+ multiply two tensors
+ multiply two tensors, but inplace
+ division of two tensors
+ matrix multiplication of two tensors

## Numpy
Sometimes we'll need to convert torch tensor to numpy array or vice versa.

In [ ]:
import numpy as np
a = np.ones(5)
print(a)
b=torch.from_numpy(a)
print(b)

### DIY
+ create torch tensor of size (5,2) containing all ones
+ convert it to numpy
+ convert it back to torch tensor

# Autograd

The ``autograd`` package plays a central role in PyTorch. Let's consider the simplest neural network, with input ``x``, parameters ``w`` and ``b``, and some loss function.

In [ ]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In this network, ``w`` and ``b`` are **parameters**, which we need to optimize. Thus, we need to be able to compute the gradients of loss
function with respect to those variables. In order to do that, we set the ``requires_grad`` property of those tensors as you can see above.

The backward propagation is stored in ``grad_fn`` property of a tensor.

In [ ]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

To optimize weights of parameters in the neural network, we need to
compute the derivatives of our loss function with respect to parameters,
namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of
``x`` and ``y``. To compute those derivatives, we call
``loss.backward()``, and then retrieve the values from ``w.grad`` and ``b.grad``:

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

By default, all tensors with ``requires_grad=True`` are tracking their
computational history and support gradient computation. However, there
are some cases when we do not need to do that, for example, when we have
trained the model and just want to apply it to some input data, i.e. we
only want to do *forward* computations through the network. We can stop
tracking computations by surrounding our computation code with
``torch.no_grad()`` block or alternatively use ``detach``.

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

# Feedforward on half moons

Run the following to generate the half moons dataset

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets

# Generate a dataset and plot it
np.random.seed(0)
num_samples = 500

X, y = sklearn.datasets.make_moons(1000, noise=0.20, random_state=1234)

# define train, validation, and test sets
X_tr = X[:num_samples].astype('float32')
X_val = X[num_samples:(num_samples+100)].astype('float32')
X_te = X[(num_samples+100):].astype('float32')

# and labels
y_tr = y[:num_samples].astype('int32')
y_val = y[num_samples:(num_samples+100)].astype('int32')
y_te = y[(num_samples+100):].astype('int32')

plt.scatter(X_tr[:,0], X_tr[:,1], s=40, c=y_tr, cmap=plt.cm.Spectral)

print(X.shape, y.shape)

num_features = X_tr.shape[-1]
num_output = 2

In [ ]:
import seaborn as sns

def show_separation(net):
    sns.set(style="white")

    xx, yy = np.mgrid[-1.5:2.5:.01, -1.:1.5:.01]
    grid = np.c_[xx.ravel(), yy.ravel()]
    batch = torch.from_numpy(grid).type(torch.float32)
    with torch.no_grad():
        probs = torch.max(F.softmax(net(batch),dim=1),dim=1).values
        probs = probs.numpy().reshape(xx.shape)

    f, ax = plt.subplots(figsize=(16, 10))
    ax.set_title("Decision boundary", fontsize=14)
    contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu",
                          vmin=0, vmax=1)
    ax_c = f.colorbar(contour)
    ax_c.set_label("$P(y = 1)$")
    ax_c.set_ticks([0, .25, .5, .75, 1])

    ax.scatter(X_te[:,0], X_te[:, 1], c=y_te, s=50,
               cmap="RdBu", vmin=-.2, vmax=1.2,
               edgecolor="white", linewidth=1)

    ax.set(xlabel="test $X_1$", ylabel="test $X_2$")

    plt.show()

Let's walk through fitting logistic regression (which is an MLP with zero hidden layers) to this (using SGD). Afterwards you'll be asked to upgrade the logistic regression model to an MLP with at least one hidden layer.

In [ ]:
import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # Setting up variables, these variables are weights in your 
        # network that can be updated while running our graph.
        # Notice, to make a hidden layer, the weights need to have the 
        # following dimensionality:
        #   W[number_of_units_going_out, number_of_units_going_in]
        #   b[number_of_units_going_out]
        # in the example below we have 2 input units (num_features) and 2 output units (num_output)
        # so our weights become W[2, 2], b[2]
        # if we want to make a hidden layer with 100 units, we need to define the shape of the
        # first weight to W[100, 2], b[2] and the shape of the second weight to W[2, 100], b[2]
        
        self.W_1 = Parameter(torch.randn(num_output, num_features)) 
        self.b_1 = Parameter(torch.randn(num_output))
        
#         hidden layer
#         self.W_2 = Parameter(torch.randn(num_output, 100)) 
#         self.b_2 = Parameter(torch.randn(num_output))
        
    def forward(self, x):
        # Read documentation on F.linear to confirm this does what we want it to do
        x = F.linear(x, self.W_1, self.b_1)
#         x = F.relu(x)
#         x = F.linear(x, self.W_2, self.b_2)

        return x # later, the activation function softmax is to be performed on the second dimension 


net = Net()

It's sometimes helpful to print a few things associated to ``net``.

In [ ]:
# list all parameters in your network
print("NAMED PARAMETERS")
print(list(net.named_parameters()))
print()
# the .parameters() method simply gives the Tensors in the list
print("PARAMETERS")
print(list(net.parameters()))
print()

# list individual parameters by name
print('WEIGHTS')
print(net.W_1)
print(net.W_1.size())
print('\nBIAS')
print(net.b_1)
print(net.b_1.size())

Can we understand better what ``Parameter`` is?

In [ ]:
param = net.W_1
print("## this is the tensor")
print(param.data)
print("\n## this is the tensor's gradient")
print(param.grad)
# notice, the gradient is undefined because we have not yet run a backward pass

print("\n## is it a leaf in the graph?")
print(param.is_leaf)

Let's define the cross entropy loss function from scratch. 

In [ ]:
def cross_entropy(ys, ts):
    # computing cross entropy per sample
    cross_entropy = -torch.sum(ts * torch.log(ys), dim=1, keepdim=False)
    # averaging over samples
    return torch.mean(cross_entropy)

Here are two helper functions, one for calculating accuracy and the other that converts outputs to one hot encoding.

In [ ]:
def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], torch.max(ts, 1)[1])
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

def onehot(t, num_classes):
    out = np.zeros((t.shape[0], num_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

To train our neural network we will use (batch) gradient descent. Minibatch SGD has many appealing properties compared to batch gradient descent. I think it's a bit easier to work with minibatches through ``DataLoader`` structure in PyTorch. So until we introduce that, let's just keep it simple with batch gradient descent. 

We can still use the SGD option in ``torch.optim`` to take care of the optimization which can handle batches that are 1) the entire dataset, 2) one data point at a time, 3) minibatches of data. 

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

Let's get to the training loop!

In [ ]:
# number of training passses
num_epochs = 1000
# store loss and accuracy for information
train_losses, val_losses, train_accs, val_accs = [], [], [], []

inputs = torch.from_numpy(X_tr)
targets = torch.from_numpy(onehot(y_tr, num_output)).float()

# training loop
for e in range(num_epochs):
    
    # zero out accumulated gradients in parameters
    optimizer.zero_grad()
    # forward pass
    output = net(inputs)
    prob = F.softmax(output, dim=1)
    # compute cross entropy loss
    loss = cross_entropy(prob, targets)
    # compute gradients given loss
    loss.backward()
    # update the parameters given the computed gradients
    optimizer.step()
    
    # the main training loop finishes above, below let's do some logging.
    
    # store training loss
    train_acc = accuracy(output, targets)
    train_losses.append(loss.data.numpy())
    train_accs.append(train_acc)
    
    # get validation inputs and expected output as torch Variables and make sure type is correct
    val_inputs = torch.from_numpy(X_val)
    val_targets = torch.from_numpy(onehot(y_val, num_output)).float()
    
    # predict with validation inputs
    with torch.no_grad():
        val_output = net(val_inputs)
        # compute loss and accuracy
        prob = F.softmax(val_output, dim=1)
        val_loss = cross_entropy(prob, val_targets)
        val_acc = accuracy(val_output, val_targets)
    
    # store loss and accuracy
    val_losses.append(val_loss.data.numpy())
    val_accs.append(val_acc.data.numpy())
    
    if e % 100 == 0:
        print("Epoch %i, "
              "Train Cost: %0.3f"
              "\tVal Cost: %0.3f"
              "\t Val acc: %0.3f" % (e, 
                                     train_losses[-1],
                                     val_losses[-1],
                                     val_accs[-1]))

The validation metrics printed above might guide you in your hyperparameter search. Instead of reading the chart above, we can also graph it for better visualization.

In [ ]:
plt.figure()
epoch = np.arange(len(train_losses))
plt.plot(epoch, train_losses, 'r', label='Train Loss')
plt.plot(epoch, val_losses, 'b', label='Val Loss')
plt.legend()
plt.xlabel('Updates')
plt.ylabel('Loss')
plt.show()

plt.figure()
plt.plot(epoch, train_accs, 'r', label='Train Acc')
plt.plot(epoch, val_accs, 'b', label='Val Acc')
plt.legend()
plt.xlabel('Updates')
plt.ylabel('Accuracy')
plt.show()

Notice we correctly put the test set into a vault. Now that we are done training, we can get the test set out of the vault and see how we really did. 

In [ ]:
# get test input and expected output
test_inputs = torch.from_numpy(X_te)
test_targets = torch.from_numpy(onehot(y_te, num_output)).float()
# predict on testset
te_output = net(test_inputs)
prob = F.softmax(te_output, dim=1)
# compute loss and accuracy
te_loss = cross_entropy(prob, test_targets)
te_acc = accuracy(te_output, test_targets)
print("\nTest Cost: %0.3f\tTest Accuracy: %0.3f" % (te_loss.data.numpy(), te_acc.data.numpy()))

show_separation(net)

## DIY
+ We used the softmax activation function in the output layer which takes a vector input and spits back a probability vector of the same dimension. The sigmoid activation function is also frequently used in the output layer. See if you can figure out what needs to be modified to run the code above with sigmoid output activation function. 
+ Create a more expressive neural network than logistic regression by inserting at least one more hidden layer.
+ In the hidden layer, experiment with more or less hidden units
+ As you increase the number of hidden layers or hidden units per layer, do you observe any overfitting?
+ Read up on the ``torch.optim`` package and upgrade SGD to momentum or adam.
+ Try minibatch SGD by using ``DataLoader``. Starter code is provided below.
+ What's the highest accuracy you can get on the test set?

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

training_set = TensorDataset(inputs, targets)
train_loader = torch.utils.data.DataLoader(training_set, batch_size=50)

val_set = TensorDataset(val_inputs, val_targets)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=50)

test_set = TensorDataset(test_inputs, test_targets)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=50)

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)

train_losses = []
for e in range(num_epochs):  # <----------- iterate over the dataset several times
    for x_batch, y_batch in train_loader:  # <------ iterate over the dataset. Since we use SGD and not GD, we take batches of a given size
        optimizer.zero_grad()  # <------------- zero out the gradients of the model
        output = net(x_batch)  # <------------- get "logits" from the model
        prob = F.softmax(output, dim=1)
        loss = cross_entropy(prob, y_batch)  # <--- calculate "loss" for logistic regression
        loss.backward()  # <------------------- calculate gradients
        optimizer.step()  
        train_losses.append(loss.detach())


plt.plot(range(len(train_losses)), train_losses)
plt.xlabel("Updates")
plt.ylabel("Loss")
plt.show()